In [0]:
from pyspark.sql import SparkSession
import pandas as pd
import random

# Initialize Spark Session
spark = SparkSession.builder.appName("Databricks_Catalog_Read").getOrCreate()

# Define the catalog and schema
catalog_name = "workspace"
schema_name = "default"

# Read data from Databricks tables and convert them to Pandas DataFrames

table_full_path = f"{catalog_name}.{schema_name}.{'user_data'}"
    
# Read the table using Spark
df_spark = spark.read.table(table_full_path)
    
# Convert to Pandas DataFrame
globals()['df_user'] = df_spark.toPandas()

In [0]:
# Show all rows
pd.set_option('display.max_rows', None)

In [0]:
df_user

In [0]:
import pandas as pd

# Print columns to debug issue
print("Available columns:", df_user.columns)

# Create 'user_name' column only if 'first_name' and 'last_name' exist
if 'first_name' in df_user.columns and 'last_name' in df_user.columns:
    df_user['user_name'] = df_user['first_name'] + " " + df_user['last_name']
    
    # Drop 'first_name' and 'last_name' only if they exist
    df_user.drop(columns=['first_name', 'last_name'], inplace=True)

# Convert 'date_joined' column to datetime format
df_user['date_joined'] = pd.to_datetime(df_user['date_joined'])

# Reorder the columns
df_user = df_user[['ind', 'user_name', 'age', 'date_joined']]


df_user

In [0]:
# Convert Pandas DataFrame to Spark DataFrame
df_user_spark = spark.createDataFrame(df_user)

#Save as managed delta table
df_user_spark.write.mode("overwrite").saveAsTable("workspace.default.df_user")